In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pickle
from numba import njit
from numba.typed import List

# Loading the graph
with open("graph.pickle", "rb") as f:
    G = pickle.load(f)

@njit
def transition_probability(t):
    """Calculate transition probability based on travel time."""
    return np.exp(-min(t, 5))  # Cap at exp(-5) to avoid underflow

@njit
def reward(t):
    """Calculate reward based on travel time."""
    return np.exp(-min(t, 5))  # Cap at exp(-5) to avoid underflow

@njit
def bellman_update(s, V, neighbors, travel_times, gamma):
    """Perform Bellman update for a single state."""
    max_value = -np.inf
    for s_next in neighbors:
        t = travel_times[s, s_next]
        value = transition_probability(t) * (reward(t) + gamma * V[s_next])
        max_value = max(max_value, value)
    return max_value

@njit
def value_iteration_numba(neighbors_list, travel_times, gamma, theta, max_iterations):
    """Perform value iteration using Numba."""
    n_nodes = len(neighbors_list)
    V = np.zeros(n_nodes)
    
    for _ in range(max_iterations):
        delta = 0
        for s in range(n_nodes):
            v = V[s]
            V[s] = bellman_update(s, V, neighbors_list[s], travel_times, gamma)
            delta = max(delta, abs(v - V[s]))
        if delta < theta:
            break
    
    return V

def prepare_graph_data(G):
    """Prepare graph data for Numba-optimized value iteration."""
    node_to_index = {node: i for i, node in enumerate(G.nodes())}
    index_to_node = {i: node for node, i in node_to_index.items()}
    n_nodes = len(G)
    
    neighbors_list = List()
    for node in G.nodes():
        neighbors_list.append(np.array([node_to_index[n] for n in G.neighbors(node)], dtype=np.int64))
    
    travel_times = np.full((n_nodes, n_nodes), np.inf)
    
    for u, v, data in G.edges(data=True):
        i, j = node_to_index[u], node_to_index[v]
        travel_times[i, j] = data['travel_time']
        travel_times[j, i] = data['travel_time']  # Assuming undirected graph
    
    return neighbors_list, travel_times, node_to_index, index_to_node

def value_iteration(G, gamma=0.9, theta=0.0001, max_iterations=1000):
    """Perform value iteration on the graph."""
    neighbors_list, travel_times, node_to_index, index_to_node = prepare_graph_data(G)
    
    V = value_iteration_numba(neighbors_list, travel_times, gamma, theta, max_iterations)
    
    optimal_values = {index_to_node[i]: v for i, v in enumerate(V)}
    
    policy = {}
    for s in G.nodes():
        s_index = node_to_index[s]
        best_action = max(
            G.neighbors(s),
            key=lambda s_next: transition_probability(G[s][s_next]['travel_time']) * (
                reward(G[s][s_next]['travel_time']) + gamma * V[node_to_index[s_next]]
            )
        )
        policy[s] = best_action
    
    return optimal_values, policy

# Run value iteration
optimal_values, optimal_policy = value_iteration(G)

# Print some results
print("Optimal Values (sample):")
for node in list(G.nodes())[:5]:
    print(f"Node {node}: {optimal_values[node]:.4f}")

print("\nOptimal Policy (sample):")
for node in list(G.nodes())[:5]:
    print(f"From Node {node} go to Node {optimal_policy[node]}")

AttributeError: 'dict' object has no attribute 'nodes'